In [1]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project Sem 2\\Data viz Project 2\\WellnessAI-Interactive-Health-Risk-Analysis\\Research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project Sem 2\\Data viz Project 2\\WellnessAI-Interactive-Health-Risk-Analysis'

In [11]:
dir_path = "Model_weights/Model_selection"

In [6]:
from ml_Project.utils.common import create_directories

In [13]:
def create_dir(dir_path):
    create_directories([dir_path])

In [14]:
create_dir(dir_path)

[2024-08-21 16:07:49,393: INFO: common: created directory at: Model_weights/Model_selection]


In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, BatchNormalization, Dropout, add
from tensorflow.keras.utils import to_categorical
import json

In [36]:
train_data_path = "Dataset/Data_transformation/train.csv"
test_data_path = "Dataset/Data_transformation/test.csv"
target_col = "Diabetes_012"
dir_path = "Model_weights/Model_selection"
history_filepath = "History"

In [37]:
import numpy as np
import pandas as pd
import json
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, add
from tensorflow.keras.utils import to_categorical

class ModelTrainer:
    def __init__(self, train_data_path, test_data_path, target_col, dir_path, history_filepath):
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.target_col = target_col
        self.dir_path = dir_path
        self.history_filepath = history_filepath

    def create_dir(self):
        create_directories([self.dir_path])
        create_directories([self.history_filepath])

    def load_data(self):
        train_data = pd.read_csv(self.train_data_path)
        test_data = pd.read_csv(self.test_data_path)
        return train_data, test_data

    def preprocess_data(self, train_data, test_data):
        X_train = train_data.drop([self.target_col], axis = 1)
        X_test = test_data.drop([self.target_col], axis = 1)
        y_train = train_data[[self.target_col]]
        y_test = test_data[[self.target_col]]

        # One-hot encode the target variable
        y_train = to_categorical(y_train, num_classes=3)
        y_test = to_categorical(y_test, num_classes=3)

        return X_train, X_test, y_train, y_test

    def build_custom_model(self, input_shape):
        # Input layer
        inputs = Input(shape=input_shape)

        # Layer 1
        x = Dense(128, activation = "relu")(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)

        # Layer 2
        x = Dense(256, activation = "relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)

        # Layer 3
        x = Dense(256, activation = "relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        
        # Output layer
        outputs = Dense(3, activation = "softmax")(x)

        # Build model
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

        return model

    def train(self):

        self.create_dir()

        # Load data
        train_data, test_data = self.load_data()

        # Preprocess data
        X_train, X_test, y_train, y_test = self.preprocess_data(train_data, test_data)

        # Build and train model
        model = self.build_custom_model((X_train.shape[1], ))
        history = model.fit(X_train, y_train, epochs = 1, batch_size = 128, validation_data = (X_test, y_test))

        # Evaluate model
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"Model Accuracy: {accuracy:.4f}")

        # Save model weights
        model.save_weights(self.dir_path + "/model_2_mlp.h5")

        # Save history
        history_dict = history.history
        history_path = os.path.join(self.history_filepath, "training_history_model_2.json")
        with open(history_path, 'w') as f:
            json.dump(history_dict, f)

        print(f"Training history saved to {history_path}")

In [38]:
try:
    trainer = ModelTrainer(train_data_path,
                           test_data_path,
                           target_col,
                           dir_path,
                           history_filepath)
    
    trainer.train()
except Exception as e:
    raise e

[2024-08-21 16:38:40,824: INFO: common: created directory at: Model_weights/Model_selection]
[2024-08-21 16:38:40,826: INFO: common: created directory at: History]
2849/2849 [==============================] - 17s 6ms/step - loss: 0.9436 - accuracy: 0.5344
Model Accuracy: 0.5344
Training history saved to History\training_history_model_2.json
